In [4]:

__author__="Irene Li <irenelizihui@gmail.com>"
__date__ ="$May 16, 2016"

import sys
from collections import defaultdict



In [49]:
class decoder():
    def __init__(self):
        self.trigram = defaultdict(float)
        self.tag = defaultdict(float)
        
    def build_maps(self,infile):
        for line in infile:
            #TAG:photobleached:I-GENE -1.0
            group = line.split(' ')
            if group[0].split(':')[0]=='TAG':
                self.tag[group[0]]=float(group[1])
            elif group[0].split(':')[0]=='TRIGRAM':
                self.trigram[group[0]]=float(group[1])
            
        print (len(self.tag),len(self.trigram))
        #print (self.tag)
        
    def get_g(self,x,tags):
        g_for_x = defaultdict(lambda : defaultdict(int))
        
        trigram_values = defaultdict(float) #g trigram <s:u:v> = 1 or 0
        tag_values = defaultdict(float) #g tag <u:r> = 1 or 0
        
        history = defaultdict(tuple) # tupel of four <t-2,t-1,x,i>
       
        
        
        #let's create the history
        tags.insert(0,'*') #init
        tags.insert(0,'*') #init
        for i in range(0,len(x)):
            history[i+1]=tuple((tags[i],tags[i+1],x))

            
        #let's create the trigram_values and tag_values
        #for each word
        for i in range(0,len(x)):
            #for all possible tags in y
            y_current = ['O',' I-GENE']
            if i == 0:
                y_sub2 = y_sub1 = ['*']
            elif i == 1:
                y_sub2 = ['*']
                y_sub1 = y_current
            else: 
                y_sub2 = y_sub1 = y_current
            
            for y_sub2_tag in y_sub2:
                for y_sub1_tag in y_sub1:
                    for y_current_tag in y_current:
                        if y_sub2_tag == tags[i] and y_sub1_tag == tags[i+1] and y_current_tag == tags[i+2]:
                            trigram_values [tuple((y_sub2_tag,y_sub1_tag,y_current_tag,(i+1)))] = 1.
                        if y_current_tag == tags[i+2]:
                            tag_values [tuple((tags[i+2],x[i]))] = 1.
        
        
        #get v * g
        for i in range(0,len(x)):
            trigram_list = ['TRIGRAM',history[i+1][0],history[i+1][1],tags[i+2]]
            v_trigram = self.trigram[':'.join(trigram_list)]
            now = tuple((history[i+1][0],history[i+1][1],tags[i+2],(i+1)))
            
            g_tag = tag_values[tuple((tags[i+2],x[i]))]
            tag_word_list = ['TAG',x[i],tags[i+2]]
            v_tag = self.tag[':'.join(tag_word_list)]
            
            score = trigram_values[now] * v_trigram + v_tag * g_tag
            print ('position i:',i,';score:',score)
        
    # pseudo code of ViterbiGLM
    # q functon is missing :(
    def ViterbiGLM(self,v,g,x):
        pi = defaultdict(float)
        pi[tuple((0,'*','*'))] = 1.
        
        for k in range(0,len(x)):
            s = ['O',' I-GENE']
            if i == 0:
                s_sub2 = s_sub1 = ['*']
            elif i == 1:
                s_sub2 = ['*']
                s_sub1 = s
            else: 
                s_sub2 = s_sub1 = s
            
            for u in s_sub1:
                for v in s:
                    max_pi = []
                    for t in s_sub2:
                        max_pi.append(pi[tuple((k-1,t,u))] * q(v|t,u,w(1:n),k))
                    pi[tuple((k,u,v))] = max(max_pi)
        
        # get tags here....
        return tag_seq            
        
    
    def load_sentence(self, train_file):
        sentence = []
        tags = []
        for line in train_file:
            if len(line.strip().split(' ')) == 2:
                line_list = line.strip().split(' ')
                sentence.append(line_list[0])
                tags.append(line_list[1])
            elif len(line.strip()) == 0:
               
                self.get_g(sentence,tags)
                sentence.clear()
                tags.clear()
        
    

In [50]:
def usage():
    sys.stderr.write("""
   Whatever...""")

if __name__ == "__main__":
    decoder = decoder()
    input_file = open (r'tag.model','r')
    train_file = open (r'small_gene.train','r')
    decoder.build_maps(input_file)
    decoder.load_sentence(train_file)
    #decoder.get_g()
    print ('start')

33090 11
position i: 0 ;score: 7.0
position i: 1 ;score: 66.0
position i: 2 ;score: 0.0
position i: 3 ;score: 0.0
position i: 4 ;score: 51.0
position i: 5 ;score: 0.0
position i: 6 ;score: 0.0
position i: 7 ;score: 0.0
position i: 0 ;score: 4.0
position i: 1 ;score: 5.0
position i: 2 ;score: 51.0
position i: 3 ;score: 2.0
position i: 4 ;score: 6.0
position i: 5 ;score: 18.0
position i: 0 ;score: 17.0
position i: 1 ;score: -12.0
position i: 2 ;score: 32.0
position i: 3 ;score: 3.0
position i: 4 ;score: 3.0
position i: 5 ;score: 47.0
position i: 6 ;score: 15.0
position i: 7 ;score: 41.0
position i: 8 ;score: 47.0
position i: 9 ;score: 20.0
position i: 10 ;score: 51.0
position i: 11 ;score: -16.0
position i: 12 ;score: 1.0
position i: 13 ;score: 59.0
position i: 14 ;score: 19.0
position i: 15 ;score: 51.0
position i: 16 ;score: 1.0
position i: 17 ;score: -8.0
position i: 18 ;score: 11.0
position i: 19 ;score: 16.0
position i: 20 ;score: 44.0
position i: 21 ;score: 18.0
position i: 22 ;sco